In [1]:
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Subset, Dataset
from torch import nn, optim
import torch
from sklearn.metrics import accuracy_score
from fastprogress.fastprogress import master_bar, progress_bar
import os
from tqdm import tqdm
from torch.nn.functional import softmax
from zipfile import ZipFile 
from PIL import Image

# Configs 

In [2]:
batch_size = 8
lr = 1e-3
tmax = 10
eta_min = 1e-5
num_epochs = 10
num_classes = 80
debug = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset Class

In [3]:
trainsforms_dict = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
    ])
}


class FreeShoundTrainDataset(Dataset):
    def __init__(self, mels, labels, transform, time_mask=0.1, freq_mask=0.1, spec_aug=True):
        self.mels = mels  # (num_samples, 128, var, 3)
        self.labels = labels
        self.tfms = transform
        self.time_mask = time_mask
        self.freq_mask = freq_mask
        self.spec_aug = spec_aug

    def __len__(self):
        return len(self.mels)

    def __getitem__(self, idx):
        mel = self.mels[idx]  # (128, 451, 3)
        base_dim, time_dim, _ = mel.shape
        crop = np.random.randint(0, time_dim-base_dim)  # (97)
        image = mel[:, crop: crop + base_dim, ...]  # (128, 128, 3)

        if self.spec_aug:
            freq_mask_begin = int(np.random.uniform(
                0, 1 - self.freq_mask) * base_dim)
            image[freq_mask_begin:freq_mask_begin +
                  int(self.freq_mask * base_dim), ...] = 0
            time_mask_begin = int(np.random.uniform(
                0, 1 - self.time_mask) * base_dim)
            image[:, time_mask_begin:time_mask_begin +
                  int(self.time_mask * base_dim), ...] = 0

        image = Image.fromarray(image[..., 0], mode='L')  # (128, 128)
        image = self.tfms(image).div_(255)  # (1, 128, 128)
        if self.labels is not None:
            label = np.asarray(self.labels)[idx]
            label = torch.from_numpy(label).float()
        return (image, label) if self.labels is not None else image

# Model 

In [4]:
import torch.nn as nn
import torchvision.models as models


class Model(nn.Module):
    def __init__(self, num_classes, pretrained=False):
        super().__init__()

        # Load the model with pretrained weights if specified
        self.model = models.resnet18(weights=None)

        # Modify the first convolution layer for grayscale input (1 channel)
        self.model.conv1 = nn.Conv2d(
            1, 64, kernel_size=7, stride=2, padding=3, bias=False
        )

        # Get the input features of the final fully connected layer
        self.in_features = self.model.fc.in_features

        # Replace the final fully connected layer
        self.model.fc = nn.Linear(self.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Utilities 

In [5]:
def transform_labels(df):
    df["labels"] = df["labels"].str.split(",")
    unique_labels = set()
    for labels in df["labels"]:
        unique_labels.update(labels)
    unique_labels = sorted(list(unique_labels))

    for label in unique_labels:
        df[label] = df["labels"].apply(lambda x: 1 if label in x else 0)

    df = df.drop("labels", axis=1)

    return df


def make_prediction(model, test_dl):
    model.eval()
    preds = []
    for xb in tqdm(test_dl):
        with torch.no_grad():
            logits = model(xb.to(device))
            pred = softmax(logits, dim=1)
        preds.extend(pred.cpu().numpy())
    return preds


def create_unique_labels(all_labels):
    label_dict = {}
    all_labels_set = []
    first_labels_set = []
    for labs in all_labels:
        lab = labs.split(",")
        for l in lab:
            if l in label_dict:
                label_dict[l] = label_dict[l] + 1
            else:
                label_dict[l] = 0

        all_labels_set.append(set(lab))
        first_labels_set.append(lab[0])
    classes = list(label_dict.keys())

    return label_dict, classes, all_labels_set, first_labels_set

In [6]:
path = Path("/kaggle/input/freesound-audio-tagging-2019")
data_dir = Path('data')
(data_dir/'test_files').mkdir(parents=True, exist_ok=True)

In [7]:
%%time
# loading the temp.zip and creating a zip object 
with ZipFile(path/'test.zip', 'r') as zObject: 
    zObject.extractall(path=data_dir/'test_files') 
zObject.close() 

CPU times: user 20.5 s, sys: 3.54 s, total: 24 s
Wall time: 33.3 s


In [8]:
# load input data
path = Path("/kaggle/input/freesound-audio-tagging-2019")
train_df = pd.read_csv(path / "train_curated.csv")
train_df = transform_labels(train_df)
y_train = train_df.iloc[:, 1:]

test_images = sorted(os.listdir("data/test_files"))
sub = pd.read_csv(path / "sample_submission.csv")

# load preprocessed audios and corresponding labels
path = Path('/kaggle/input/data-preprocessing/data')
processed_test = pickle.load(
    open(path / "mels_test.pkl", "rb")
)  # (4970, 128, var, 3)
y_train = train_df.iloc[:, 1:]


test_ds = FreeShoundTrainDataset(processed_test, None, trainsforms_dict["train"])
if debug:
    test_ds = Subset(test_ds, range(100))
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False)
print("=======Data Loader Prepared=========")

# load model
model = Model(num_classes=80)
model.load_state_dict(torch.load("/kaggle/input/sound-classification-training/model.pt",weights_only=True, map_location=device))
model.to(device)
print("=======Model Loaded=========")

preds = make_prediction(model, test_dl)
sub = pd.DataFrame(preds, columns=y_train.columns)
if debug:
    test_fns = sorted(os.listdir("data/test_files"))[:100]
else:
    test_fns = sorted(os.listdir("data/test_files"))
sub["fname"] = test_fns
sub = sub[["fname"] + [col for col in sub.columns if col != "fname"]]
sub.to_csv('submission.csv', index=False)

=======Data Loader Prepared=========
=======Model Loaded=========


100%|██████████| 421/421 [00:04<00:00, 103.71it/s]


In [9]:
!rm -rf data/

In [10]:
sub.shape

(3361, 81)

In [11]:
sub.head()

,fname,Accelerating_and_revving_and_vroom,Accordion,Acoustic_guitar,Applause,Bark,Bass_drum,Bass_guitar,Bathtub_(filling_or_washing),Bicycle_bell,...,Toilet_flush,Traffic_noise_and_roadway_noise,Trickle_and_dribble,Walk_and_footsteps,Water_tap_and_faucet,Waves_and_surf,Whispering,Writing,Yell,Zipper_(clothing)
0,4260ebea.wav,0.000496,6.441688e-05,2.970595e-04,0.000742,0.000221,8.809141e-03,6.113598e-04,0.037526,0.038531,...,0.001020,0.000885,0.011949,0.006969,0.021221,0.001245,0.017028,0.016149,0.000395,0.005170
1,426eb1e0.wav,0.000060,6.506274e-07,8.909593e-07,0.646719,0.000036,2.223978e-06,6.902122e-07,0.000799,0.000002,...,0.000821,0.000743,0.000319,0.000101,0.000035,0.001049,0.000248,0.000008,0.000099,0.000128
2,428d70bb.wav,0.003783,5.519008e-03,4.867644e-04,0.002592,0.000930,9.883742e-05,4.452369e-05,0.002924,0.004953,...,0.002445,0.027090,0.002303,0.001165,0.000831,0.002758,0.011902,0.000948,0.004483,0.012611
3,4292b1c9.wav,0.000004,4.029113e-06,1.252604e-06,0.000026,0.000012,4.401098e-07,5.743286e-08,0.000062,0.000008,...,0.000002,0.000033,0.000098,0.000040,0.000002,0.000048,0.005773,0.000124,0.010912,0.001816
4,429c5071.wav,0.020863,1.449897e-04,5.794917e-04,0.001187,0.009021,2.688271e-03,6.961418e-04,0.001832,0.000853,...,0.006168,0.001560,0.000401,0.140091,0.001587,0.007004,0.001136,0.017590,0.000271,0.122457
